In [33]:
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
from langchain_teddynote import logging

logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


In [3]:
import pandas as pd
df = pd.read_csv("data/ragas_synthetic_dataset.csv")
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What role does content moderation play in the ...,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",Content moderation plays a crucial role in AI ...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
1,What are the key aspects of the AI Development...,"['SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스...",The key aspects of the AI Development Guidelin...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
2,What was the purpose of the AI Safety Summit h...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 AI 안...,The purpose of the AI Safety Summit held in No...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
3,What does 'Operationalizing Progress' refer to...,"['구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY ...",The term 'Operationalizing Progress' in the co...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
4,What improvements does Tongyi Qianwen 2.0 have...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라...,Tongyi Qianwen 2.0 has improvements over the p...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True


In [4]:
df.shape

(10, 6)

In [5]:
# Dataset으로 wrapping이 되어야 ragas를 할 수있다.
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})

In [6]:
import ast

# contexts를 list형식으로 바꿔준다. 원래는 문자열이다.
def convert_to_list(example):
    contexts = ast.literal_eval(example["contexts"])
    return {"contexts": contexts}

test_dataset = test_dataset.map(convert_to_list)
print(test_dataset)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})


In [7]:
test_dataset[1]["contexts"]

['SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의\nKEY Contents\nn G7이 첨단 AI 시스템을 개발하는 기업을 대상으로 AI 위험 식별과 완화를 위해 자발적인\n채택을 권고하는 AI 국제 행동강령을 마련\nn 행동강령은 AI 수명주기 전반에 걸친 위험 평가와 완화, 투명성과 책임성의 보장, 정보공유와\n이해관계자 간 협력, 보안 통제, 콘텐츠 인증과 출처 확인 등의 조치를 요구\n£G7, 첨단 AI 시스템의 위험 관리를 위한 국제 행동강령 마련\nn 주요 7개국(G7)*은 2023년 10월 30일 ‘히로시마 AI 프로세스’를 통해 AI 기업 대상의 AI 국제\n행동강령(International Code of Conduct for Advanced AI Systems)에 합의\n∙ G7은 2023년 5월 일본 히로시마에서 개최된 정상회의에서 생성 AI에 관한 국제규범 마련과\n정보공유를 위해 ‘히로시마 AI 프로세스’를 출범**\n∙ 기업의 자발적 채택을 위해 마련된 이번 행동강령은 기반모델과 생성 AI를 포함한 첨단 AI 시스템의\n위험 식별과 완화에 필요한 조치를 포함\n* 주요 7개국(G7)은 미국, 일본, 독일, 영국, 프랑스, 이탈리아, 캐나다를 의미\n** 5월 정상회의에는 한국, 호주, 베트남 등을 포함한 8개국이 초청을 받았으나, AI 국제 행동강령에는 우선 G7 국가만 포함하여 채택\nn G7은 행동강령을 통해 아래의 조치를 제시했으며, 빠르게 발전하는 기술에 대응할 수 있도록\n이해관계자 협의를 통해 필요에 따라 개정할 예정\n∙ 첨단 AI 시스템의 개발 과정에서 AI 수명주기 전반에 걸쳐 위험을 평가 및 완화하는 조치를 채택하고,\n첨단 AI 시스템의 출시와 배포 이후 취약점과 오용 사고, 오용 유형을 파악해 완화\n∙ 첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알리는 방법으로 투명성을\n보장하고 책임성을 강화']

# RAG평가

In [15]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

loader = PDFPlumberLoader("./data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = FAISS.from_documents(documents, embeddings)

retriever = vector_store.as_retriever()

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
     | prompt
     | llm
     | StrOutputParser()
)

**분산처리하는 batch**

`batch()` 를 호출하여 배치 데이터셋에 대한 답변을 받습니다. 다량의 질문을 한번에 처리할때 용이합니다. 


In [16]:
#batch_dataset에 question을 넣어줌
batch_dataset = [question for question in test_dataset["question"]]
batch_dataset[:3]

['What role does content moderation play in the context of AI development?',
 'What are the key aspects of the AI Development Guidelines established by the G7?',
 'What was the purpose of the AI Safety Summit held in November 2023?']

In [18]:
answer = chain.batch(batch_dataset)
answer[:3]

['Content moderation plays a crucial role in the context of AI development by ensuring responsible use and deployment of AI-generated content. It involves implementing guidelines and policies to identify and manage AI-generated materials, particularly those that may mislead or harm users. For instance, platforms like YouTube are introducing mandatory labeling for AI-generated content to enhance transparency and protect users from misinformation. Additionally, content moderation helps mitigate risks associated with privacy violations and the potential for harmful or misleading representations, thereby fostering a safer digital environment. This approach aligns with broader AI governance strategies that emphasize accountability, security, and ethical standards in AI applications.',
 'The key aspects of the AI Development Guidelines established by the G7 include:\n\n1. **Risk Assessment and Mitigation**: The guidelines recommend voluntary adoption by companies to identify and mitigate ris

llm이 생성한 답변울 "answer" 컬럼에 저장

In [19]:
# "answer" 컬럼 덮어쓰기 또는 추가
if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
else:
    test_dataset = test_dataset.add_column("answer", answer)

c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\datasets\table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [21]:
# answer가 추가된건을 확인할 수 있다.
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', 'answer'],
    num_rows: 10
})

In [25]:
pd.DataFrame(test_dataset[:3])

,question,contexts,ground_truth,evolution_type,metadata,episode_done,answer
0,What role does content moderation play in the ...,"[£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\nn...",Content moderation plays a crucial role in AI ...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,Content moderation plays a crucial role in the...
1,What are the key aspects of the AI Development...,"[SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스를...",The key aspects of the AI Development Guidelin...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,The key aspects of the AI Development Guidelin...
2,What was the purpose of the AI Safety Summit h...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 AI 안전...,The purpose of the AI Safety Summit held in No...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,The purpose of the AI Safety Summit held in No...


# 답변 평가

In [26]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision
)

result = evaluate(
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall
    ]
)
result

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

{'context_precision': 0.8000, 'faithfulness': 0.5326, 'answer_relevancy': 0.8946, 'context_recall': 0.8500}

In [27]:
result_df = result.to_pandas()
result_df.head()

c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\datasets\table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
c:\Users\skyop\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-gLkynrUQ-py3.11\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


,question,contexts,answer,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,What role does content moderation play in the ...,"[£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\nn...",Content moderation plays a crucial role in the...,Content moderation plays a crucial role in AI ...,1.0,0.400000,0.994678,1.0
1,What are the key aspects of the AI Development...,"[SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스를...",The key aspects of the AI Development Guidelin...,The key aspects of the AI Development Guidelin...,1.0,0.277778,1.000000,1.0
2,What was the purpose of the AI Safety Summit h...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 AI 안전...,The purpose of the AI Safety Summit held in No...,The purpose of the AI Safety Summit held in No...,1.0,0.666667,1.000000,1.0
3,What does 'Operationalizing Progress' refer to...,"[구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY C...",'Operationalizing Progress' in the context of ...,The term 'Operationalizing Progress' in the co...,1.0,0.444444,1.000000,0.0
4,What improvements does Tongyi Qianwen 2.0 have...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라우...,Tongyi Qianwen 2.0 has improvements in underst...,Tongyi Qianwen 2.0 has improvements over the p...,1.0,0.900000,0.991582,0.5


In [28]:
result_df.to_csv("data/ragas_evaluation_result.csv", index=False)

In [30]:
result_df.loc[:, "context_precision": "context_recall"]

,context_precision,faithfulness,answer_relevancy,context_recall
0,1.0,0.400000,0.994678,1.0
1,1.0,0.277778,1.000000,1.0
2,1.0,0.666667,1.000000,1.0
3,1.0,0.444444,1.000000,0.0
4,1.0,0.900000,0.991582,0.5
5,0.0,0.000000,0.000000,1.0
6,0.0,1.000000,0.997150,1.0
7,1.0,0.818182,0.991400,1.0
8,1.0,0.583333,0.974406,1.0
9,1.0,0.235294,0.996541,1.0


# ContextRecall

검색된 context가 llm이 생성한 답변과 얼마나 일치하는지 측정.

이상적인 시나리어에서는 ground truth답변의 모든 주장이 검색된 context에 귀속될 수 있어야 한다.

# Context Precision
"얼마나 관련성 있는 문서가 상위에 배치되었는가?" 를 평가하는 지표

recall과 같이 봐야한다.
10개의 문서에서 retriever를 한다고 할때, 답변에 필요한 문서가 10개의 모든문서에 포함되어 있다고 한다면, recall은 만점일것이다.

하지만 precision관점에서는 상위에 얼마만큼 배치가 되어있는지가 중요하다.

k=10개를 검출한다고 답변이 3개가 된다고 할때 유사도가 1,2,3등을 하면 만점이다.

이상적인 시나리오는 모든 관련 chunks가 상위 문서에 나타나야 한다.

# Answer Relevancy
"생성된 답변이 주어진 prompt에 얼마나 적절한지"를 평가하는 지표

질문-답변 시스템의 성능을 평가하는데 유용하며, 생성된 답변이 원래 질문의 의도를 얼마나 잘 반영하는지를 측정

질문에 대한 임베딩과 생성된 답변에 대한 임베딩의 코사인 유사도를 측정한다. 즉 답변이 질문에 대해서 연관성이 높아야한다.

# Faithfulness
"생성된 답변의 사실적 일관성을 주어진 컨텍스트와 비교하여 측정"하는 지표